In [1]:
import elasticsearch

In [2]:
import pprint

In [10]:
app_qry = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
      "filter": {"term": {"provider_code": "aws"}},
      "aggs": {
        "account_filter": {
            "filter": {"term": {"provider_account": "123"}}
            
        }
      }
    }
  }
}

## Group by provider without context filter 

In [28]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
    "sort" : [
        { "assetName" : {"missing" : "_last"}}
    ],
    "query" : {
        "term" : { "providerName" : "1Million_data_24june" }
    }
}

import time
start_time = time.time()
res = es.search(index="gpd_one_million_new", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.021749019622802734 seconds ---


In [29]:
res

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

## Groupby provider with context filter 

In [44]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
        "terms": {"field": "provider_code"},
        "aggs": {
            "account_filter": {
                "terms": {"field": "provider_account"},
                "aggs": {
                    "application_filter": {
                        "terms": {"field": "context.application"}
                    }
                }
            
             }
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.305433988571167 seconds ---


In [20]:
res

{'took': 783,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'provider_filter': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'aws',
     'doc_count': 800000,
     'account_filter': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': '123',
        'doc_count': 200000,
        'application_filter': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0,
         'buckets': [{'key': 'finance_app', 'doc_count': 29002},
          {'key': 'd2ops', 'doc_count': 28786},
          {'key': 'ecommerce_app', 'doc_count': 28784},
          {'key': 'cam', 'doc_count': 28666},
          {'key': 'aiops', 'doc_count': 28492},
          {'key': 'store', 'doc_count': 28231},
          {'key': 'cloud_based_app', 'doc_count': 28039}]}},
       {'key': '34566'

## Group by context app and env

In [22]:
PUT cloud_resources/_mapping
{
  "properties": {
    "context.environment": { 
      "type":     "text",
      "fielddata": true
    }
  }
}

SyntaxError: invalid syntax (<ipython-input-22-9599a5828961>, line 1)

In [23]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
        "terms": {"field": "provider_code"},
        "aggs": {
            "account_filter": {
                "terms": {"field": "provider_account"},
                "aggs": {
                    "application_filter": {
                        "terms": {"field": "context.application"},
                        "aggs":{
                            "environment_filter": {
                                "terms": {"field": "context.environment"}
                            }
                        }
                    }
                }
            
             }
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.4140238761901855 seconds ---


In [24]:
res

{'took': 2251,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'provider_filter': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'aws',
     'doc_count': 800000,
     'account_filter': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': '123',
        'doc_count': 200000,
        'application_filter': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0,
         'buckets': [{'key': 'finance_app',
           'doc_count': 29002,
           'environment_filter': {'doc_count_error_upper_bound': 0,
            'sum_other_doc_count': 0,
            'buckets': [{'key': 'dev', 'doc_count': 9815},
             {'key': 'qa', 'doc_count': 9622},
             {'key': 'uat', 'doc_count': 9565}]}},
          {'key': 'd2ops',
           'doc_count': 2878

In [13]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])

query = {
  "size": 0,
#   "query": {
#     "bool": {
#       "must": [
#         { "match": { "provider_code": "TestProvider"}
#         }
#       ],
#       "filter": [
#         { "terms":  { "context.application": ["cam", "aiops"] }},
#         { "terms":  { "context.environment": ["dev", "uat"] }}
#       ]
#     }
#   },
  "aggs" : {
    "data": {
      "composite" : {
        "size": 10000,
        "sources": [
          {"providerName": {"terms": {"field": "providerName.keyword"}}},
#           {"assetType": { "terms": {"field": "assetType.keyword" } } },
#           {"app": { "terms": {"field": "context.application.keyword" } } },
#           {"env": { "terms": {"field": "context.environment.keyword" } } }
          ]
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="gpd_one_million_new", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.014163970947265625 seconds ---


In [14]:
res


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'data': {'after_key': {'providerName': '1Million_data_24june'},
   'buckets': [{'key': {'providerName': '1Million_data_24june'},
     'doc_count': 600000}]}}}

In [40]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])

query = {
  "size": 0,
  "query": {
    "bool": {
      "must": [
        { "match": { "provider_code": "TestProvider"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["cam", "aiops"] }},
        { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  },
  "aggs" : {
    "data": {
      "composite" : {
        "size": 10000,
        "sources": [
          {"provider_code": {"terms": {"field": "provider_code.keyword"}}},
          {"type": { "terms": {"field": "type.keyword" } } },
          {"app": { "terms": {"field": "context.application.keyword" } } },
          {"env": { "terms": {"field": "context.environment.keyword" } } }
          ]
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.02229619026184082 seconds ---


In [37]:
res


{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'data': {'after_key': {'provider_code': 'TestProvider',
    'type': 'service_instance',
    'app': 'cam',
    'env': 'uat'},
   'buckets': [{'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'dev'},
     'doc_count': 5531},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'uat'},
     'doc_count': 5319},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'cam',
      'env': 'dev'},
     'doc_count': 4994},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'cam',
      'env': 'uat'},
     'doc_count': 5483},
    {'key': {'provider_code': 'TestProvider',
 

## Sorting on elastic search 

In [15]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])


In [20]:
query = {
  "size": 50,
    "query": {
    "bool": {
      "must": [
        { "match": { "providerName": "1Million_data_24june"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["app1", "app2", "app3", "app4", "app5"] }}
#         { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  }
}

import time
start_time = time.time()
res = es.search(index="gpd_one_million_new", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.12983083724975586 seconds ---


In [21]:
res

{'took': 102,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 8.333323e-07,
  'hits': [{'_index': 'gpd_one_million_new',
    '_type': 'gpd_one_million_new',
    '_id': 'E-DRnGsBVD6UQdE_Tl89',
    '_score': 8.333323e-07,
    '_source': {'providerAssetId': 'subscriptions_1Million_data_24june_ba0_1Million_data_24june_ba0_aa0_resourceGroupsdefault_1Million_data_24june_6377',
     'providerName': '1Million_data_24june',
     'extractDate': '2019-02-02 01:30:00.267',
     'assetName': '861australiaeastCache',
     'assetStartDate': '2019-02-08 01:30:00.324',
     'assetEndDate': '',
     'assetCreatedBy': '',
     'assetEndedBy': '',
     'capacityText': '',
     'assetType': 'Cache',
     'providerApiText': {'sku': {'tier': 'Premium', 'name': 'Premium_LRS'},
      'kind': 'Cache',
      'name': '7404australiaeastCache',
      'tags': [],
      'id': '1Million_data_24june_ba0_aa0

In [ ]:
query = {
  "size": 10,
    "query": {
    "bool": {
      "must": [
        { "match": { "provider_code": "TestProvider"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["cam", "aiops"] }},
        { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  }
}

import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))